# Pytorch cheet sheet

to cover:refine_names, split, rename, align ... norm, lu
https://github.com/pytorch/pytorch/blob/main/torch/_tensor.py#L872
<br>
linear algebra: <br>
triu ,tril, print(x_data.trace())

https://pytorch.org/docs/stable/tensor_view.html

In [1]:
import torch as t
import numpy as np

## Create a tensor
Tensor is an array, like numpy array, but you can run it on GPUs, TPUs and other accelerating hardwares.

You can simply create a tensor from numpy array or a list

In [2]:
data = [[[1, 2, 3], [4, 6, 8]],[[3, 5, 1], [7, 9, 2]]]
x_data = t.tensor(data)
x_data

tensor([[[1, 2, 3],
         [4, 6, 8]],

        [[3, 5, 1],
         [7, 9, 2]]])

***
With `dtype` and `shape` properties you can access type and shape of a tensor. <br>
As a newbie you will time to time have headache with tensors being on the wrong device, the `device` attribute will help you locate your data.

In [3]:
print(x_data.dtype)
print(x_data.shape)
print(x_data.device)

torch.int64
torch.Size([2, 2, 3])
cpu


Another useful method is `dim` which returns the dimension of a tensor.

In [4]:
x_data.dim()

3

***
There are some familiar properties like `len` and also method like `iter`. (We will talk about indexing and slicing later)

In [5]:
print(len(x_data))
print(len(x_data[0]))

2
2


In [6]:
g = iter(x_data)
print(next(g))
print(next(g))

tensor([[1, 2, 3],
        [4, 6, 8]])
tensor([[3, 5, 1],
        [7, 9, 2]])


***
tensor can have different types, you can change the type of tensor using `to` method. You can use the same method to move tensor between gpu and cpu.

In [7]:
x = x_data.to(dtype=t.float64)
x.to('cpu') # or .to('cuda:0') .to('cuda',0)

tensor([[[1., 2., 3.],
         [4., 6., 8.]],

        [[3., 5., 1.],
         [7., 9., 2.]]], dtype=torch.float64)

### Create a special tensor
We use special tensors like, ones, zeros and ... a lot! lets review them

In [8]:
shape = [2, 3,]
rand_tensor = t.rand(shape)
ones_tensor = t.ones(shape)
zeros_tensor = t.zeros(shape)
# eye accepts one input as the output is square tensor
eye_tensor = t.eye(shape[0])
print(rand_tensor)
print(ones_tensor)
print(zeros_tensor)
print(eye_tensor)

tensor([[0.9000, 0.1991, 0.2910],
        [0.0664, 0.7480, 0.1320]])
tensor([[1., 1., 1.],
        [1., 1., 1.]])
tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([[1., 0.],
        [0., 1.]])


***
we can also use `{zeros|ones}_like` method, which create a tensor with the same shape as the input tensor

In [9]:
like_1 = t.ones_like(x_data)
print(like_1)
print(like_1.dtype)

tensor([[[1, 1, 1],
         [1, 1, 1]],

        [[1, 1, 1],
         [1, 1, 1]]])
torch.int64


for more tensor creation methods refer to pytorch's official __[documentation](https://pytorch.org/docs/stable/torch.html#creation-ops)__

## Indexing and slicing
Lets create a 2D tensor and play with slice and index. Lets use `linspace` and `reshape` to get familiar with new methods. <br>
You will see that indexing in pytorch is very similar to numpy.

In [10]:
d = t.linspace(0,8,9)
print(d)
x_data = d.reshape((3,3))
print(x_data)

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8.])
tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])


In [11]:
x_data[:,1]

tensor([1., 4., 7.])

In [12]:
x_data[1, :]

tensor([3., 4., 5.])

In [13]:
x_data[0,1]

tensor(1.)

## Views
Views are not new tensors

In [14]:
x_tensor = t.rand(3, 4)
print(x_tensor)
x_view = x_tensor.view(2,6)
print(x_view)

tensor([[0.5341, 0.0632, 0.9589, 0.4748],
        [0.7408, 0.8178, 0.1428, 0.6602],
        [0.4033, 0.7835, 0.6792, 0.4182]])
tensor([[0.5341, 0.0632, 0.9589, 0.4748, 0.7408, 0.8178],
        [0.1428, 0.6602, 0.4033, 0.7835, 0.6792, 0.4182]])


***
As you see here, both view and original tensors (despite their different shapes) point to the same locations in the memory. <br>
While `clone` of a tensor will create a copy and points to a different location in memory. 

In [15]:
print(x_tensor.untyped_storage().data_ptr())
print(x_view.untyped_storage().data_ptr())

140625088379520
140625088379520


In [16]:
y_tensor = x_tensor.clone()
print(y_tensor.untyped_storage().data_ptr())

140625088593344


## Example: Load a batch of images